# Regression Exercise - Solutions

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [5]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [6]:
y_val = housing['medianHouseValue']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
import tensorflow as tf

In [16]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [17]:
feat_cols = [age,rooms,bedrooms,pop,households,income]

**Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [18]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

**Create the estimator model. Use a DNNRegressor. Play around with the hidden units!**

In [19]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplcf9arce', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efc70ca1d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [20]:
model.train(input_fn=input_func,steps=25000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmplcf9arce/model.ckpt.
INFO:tensorflow:loss = 590735500000.0, step = 1
INFO:tensorflow:global_step/sec: 540.903
INFO:tensorflow:loss = 595270570000.0, step = 101 (0.187 sec)
INFO:tensorflow:global_step/sec: 525.014
INFO:tensorflow:loss = 536966000000.0, step = 201 (0.189 sec)
INFO:tensorflow:global_step/sec: 556.266
INFO:tensorflow:loss = 409129600000.0, step = 301 (0.180 sec)
INFO:tensorflow:global_step/sec: 538.113
INFO:tensorflow:loss = 586836100000.0, st

INFO:tensorflow:global_step/sec: 525.736
INFO:tensorflow:loss = 82311350000.0, step = 6701 (0.189 sec)
INFO:tensorflow:global_step/sec: 519.254
INFO:tensorflow:loss = 112791630000.0, step = 6801 (0.192 sec)
INFO:tensorflow:global_step/sec: 507.111
INFO:tensorflow:loss = 85363294000.0, step = 6901 (0.198 sec)
INFO:tensorflow:global_step/sec: 555.703
INFO:tensorflow:loss = 102753530000.0, step = 7001 (0.180 sec)
INFO:tensorflow:global_step/sec: 555.82
INFO:tensorflow:loss = 89173560000.0, step = 7101 (0.179 sec)
INFO:tensorflow:global_step/sec: 562.444
INFO:tensorflow:loss = 90768110000.0, step = 7201 (0.179 sec)
INFO:tensorflow:global_step/sec: 555.215
INFO:tensorflow:loss = 66341740000.0, step = 7301 (0.181 sec)
INFO:tensorflow:global_step/sec: 510.733
INFO:tensorflow:loss = 69656960000.0, step = 7401 (0.194 sec)
INFO:tensorflow:global_step/sec: 576.97
INFO:tensorflow:loss = 87198350000.0, step = 7501 (0.174 sec)
INFO:tensorflow:global_step/sec: 534.453
INFO:tensorflow:loss = 554134940

INFO:tensorflow:global_step/sec: 555.552
INFO:tensorflow:loss = 97987130000.0, step = 14601 (0.178 sec)
INFO:tensorflow:global_step/sec: 563.57
INFO:tensorflow:loss = 141884980000.0, step = 14701 (0.178 sec)
INFO:tensorflow:global_step/sec: 565.068
INFO:tensorflow:loss = 52160963000.0, step = 14801 (0.179 sec)
INFO:tensorflow:global_step/sec: 574.554
INFO:tensorflow:loss = 38317765000.0, step = 14901 (0.173 sec)
INFO:tensorflow:global_step/sec: 538.531
INFO:tensorflow:loss = 76594470000.0, step = 15001 (0.190 sec)
INFO:tensorflow:global_step/sec: 545.934
INFO:tensorflow:loss = 45640008000.0, step = 15101 (0.180 sec)
INFO:tensorflow:global_step/sec: 567.32
INFO:tensorflow:loss = 73753936000.0, step = 15201 (0.175 sec)
INFO:tensorflow:global_step/sec: 573.303
INFO:tensorflow:loss = 66627670000.0, step = 15301 (0.174 sec)
INFO:tensorflow:global_step/sec: 550.941
INFO:tensorflow:loss = 113181070000.0, step = 15401 (0.181 sec)
INFO:tensorflow:global_step/sec: 547.747
INFO:tensorflow:loss = 

INFO:tensorflow:global_step/sec: 551.098
INFO:tensorflow:loss = 96834850000.0, step = 22501 (0.181 sec)
INFO:tensorflow:global_step/sec: 548.476
INFO:tensorflow:loss = 32894083000.0, step = 22601 (0.182 sec)
INFO:tensorflow:global_step/sec: 542.036
INFO:tensorflow:loss = 61317874000.0, step = 22701 (0.184 sec)
INFO:tensorflow:global_step/sec: 509.706
INFO:tensorflow:loss = 91130310000.0, step = 22801 (0.196 sec)
INFO:tensorflow:global_step/sec: 537.095
INFO:tensorflow:loss = 72321060000.0, step = 22901 (0.185 sec)
INFO:tensorflow:global_step/sec: 544.505
INFO:tensorflow:loss = 83168715000.0, step = 23001 (0.184 sec)
INFO:tensorflow:global_step/sec: 540.011
INFO:tensorflow:loss = 40814547000.0, step = 23101 (0.186 sec)
INFO:tensorflow:global_step/sec: 549.149
INFO:tensorflow:loss = 68690833000.0, step = 23201 (0.181 sec)
INFO:tensorflow:global_step/sec: 521.521
INFO:tensorflow:loss = 158148360000.0, step = 23301 (0.192 sec)
INFO:tensorflow:global_step/sec: 518.11
INFO:tensorflow:loss = 

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [30]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y = y_test ,
    
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [22]:
pred_gen = model.predict(predict_input_func)

In [23]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmplcf9arce/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [32]:
predictions

[{'predictions': array([239182.58], dtype=float32)},
 {'predictions': array([315990.5], dtype=float32)},
 {'predictions': array([214517.19], dtype=float32)},
 {'predictions': array([188170.25], dtype=float32)},
 {'predictions': array([282971.4], dtype=float32)},
 {'predictions': array([199144.33], dtype=float32)},
 {'predictions': array([226353.48], dtype=float32)},
 {'predictions': array([207210.78], dtype=float32)},
 {'predictions': array([220981.4], dtype=float32)},
 {'predictions': array([198335.23], dtype=float32)},
 {'predictions': array([205789.88], dtype=float32)},
 {'predictions': array([223102.94], dtype=float32)},
 {'predictions': array([193661.77], dtype=float32)},
 {'predictions': array([179815.72], dtype=float32)},
 {'predictions': array([262139.06], dtype=float32)},
 {'predictions': array([180477.06], dtype=float32)},
 {'predictions': array([201425.61], dtype=float32)},
 {'predictions': array([189778.86], dtype=float32)},
 {'predictions': array([182770.97], dtype=float32

** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [24]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_squared_error(y_test,final_preds)**0.5

98621.52150996288

In [31]:
model.evaluate(predict_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-30-15:24:58
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmplcf9arce/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-30-15:24:59
INFO:tensorflow:Saving dict for global step 25000: average_loss = 9726206000.0, global_step = 25000, label/mean = 207662.89, loss = 97136560000.0, prediction/mean = 211275.83
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 25000: /tmp/tmplcf9arce/model.ckpt-25000


{'average_loss': 9726206000.0,
 'label/mean': 207662.89,
 'loss': 97136560000.0,
 'prediction/mean': 211275.83,
 'global_step': 25000}

# Great Job!